### Abstract

This notebook is intended to practise some advanced features in Python 3. 

### Context Manager


In [19]:
class MyContext(object):

    def __init__(self):
        self._mc = "my context"
    
    def __do(self):
        pass
    
    ####
    def __enter__(self):
        print("enter my context")
        return self._mc
    
    def __exit__(self, exectype, exception, traceback):
        if (traceback == None):  # no traceback means no exception
            print('No exception')
        else:
            print(exception)
        
        print('exit my context')
        

In [23]:
cxt = MyContext()

with cxt as mycontext:
    print("within the context")
    #raise Exception('some exception')

    print(mycontext)


enter my context
within the context
my context
No exception
exit my context


In [25]:
import sys
print(sys.path)

['', '/mnt/sdb/anaconda3/envs/pytorch/lib/python36.zip', '/mnt/sdb/anaconda3/envs/pytorch/lib/python3.6', '/mnt/sdb/anaconda3/envs/pytorch/lib/python3.6/lib-dynload', '/mnt/sdb/anaconda3/envs/pytorch/lib/python3.6/site-packages', '/mnt/sdb/anaconda3/envs/pytorch/lib/python3.6/site-packages/torchvision-0.1.9-py3.6.egg', '/mnt/sdb/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/extensions', '/remote/users/lguo/.ipython']


### Iterator 

In [37]:
class Fibonacci:
    def __init__(self, n = None):
        self._n = n
        elements = [1, 1]
        
        if (n == None):
            n = 2

        self._elements = elements


    def __iter__(self):
        #return iter(self._elements)
        return Suite(self._elements, self._n)
    

class Suite():
    """
        It is better to separate the iteration states from the sequence class itself.
    """
    def __init__(self, elements, n):
        self._elements = elements
        self._n = n
        self._next_index = 0

    
    def __next__(self):
        if (self._next_index >= self._n):
            raise StopIteration
        
        if (self._next_index >= len(self._elements) - 1):
            self._elements.append(self._elements[-1] + self._elements[-2])
        curr = self._elements[self._next_index] 
        self._next_index += 1
        return curr


n = 2
f = Fibonacci(4)

for i in f:
    print(i)
    
print(list(Fibonacci(10)))

1
1
2
3
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


### Generator


In [148]:
def fibonacci_gen(n):
    """
     A generator function
    """
    elem_1 = None
    elem_2 = None
    count = 0
    
    while(count < n):
        
        if (elem_1 is None or elem_2 is None):
            out = 1
        else:
            out = elem_1 + elem_2
    
        # function would be paused and resumed from here, at each next() step
        yield out
    
        count += 1
        elem_1 = elem_2
        elem_2 = out

In [157]:
fgen = fibonacci_gen(3)
print(type(fgen))
print(next(fgen))

<class 'generator'>
1


In [64]:
f = (x * 2 for x in range(10))

f

<generator object <genexpr> at 0x7f71d047b728>

In [155]:
from itertools import islice

for i in islice(fibonacci_gen(7), 7):
    print(i)

1
1
2
3
5
8
13


### Decorator


In [147]:
#global_cache = {}

def cached(func):
    """
        A decorator function to cache the results of a function call, 
          and use the result for the later call.
    """
    cache = {}  # a closure that is associated with the internal function
    
    def wrapper(*args, **kwargs):
        #args_list = [arg for arg in args]
        #print('args', str(args), 'kwargs', str(kwargs))
        
        # concatenate the normal arguments and named arguments
        cache_key = args + tuple(kwargs)
        
        if (cache_key not in cache):
            print('cache miss')
            cache[cache_key] = func(*args, **kwargs)
        
        return cache[cache_key]
    
    return wrapper


class cache_with_class():
    def __init__(self, func):
        self._cache = {}
        self._func = func

    # overload the '()' operator
    def __call__(self, *args, **kwargs):
        cache_key = args + tuple(kwargs)
        
        if (cache_key not in self._cache):
            print('cache miss')
            self._cache[cache_key] = self._func(*args, **kwargs)
        
        return self._cache[cache_key]


In [132]:
@cached
def add(a, b):
    return a + b

def increment(x):
    return x + 1

In [129]:
print('first call:', add(1, 2))
print('second call:', add(1, 2))

cache miss
first call: 3
second call: 3


In [140]:
# this is equivalent to add the @cached decorator to the increment() function
cached_increment = cached(increment)

print('first call:', cached_increment(3))
print('second call:', cached_increment(3))

cache miss
first call: 4
second call: 4


In [145]:
cached_increment = cache_with_class(increment)

cached_increment(3)

cache miss


4

In [109]:
class A:
    @staticmethod
    def add(a, b):
        return a + b

a = A()
a.add(1, 2) == A.add(1, 2)

True